In [1]:
import numpy as np
import pandas as pd
import ast
ast.literal_eval

<function ast.literal_eval(node_or_string)>

In [2]:
credits=pd.read_csv("tmdb_5000_credits.csv")
movies=pd.read_csv("tmdb_5000_movies.csv")

In [3]:
movies=movies.merge(credits,on="title")

In [4]:
# needed coloumns
# genres
# id
# keywords
# title
# overvies
# cast 
# crew

movies=movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [5]:
movies.dropna(inplace=True)

In [6]:
def convert(obj):
    l=[]
    for i in ast.literal_eval(obj):
        l.append(i['name'])
    return l    

In [7]:
movies['genres']=movies['genres'].apply(convert)


In [8]:
movies['keywords']=movies['keywords'].apply(convert)

In [9]:
def convert3(obj):
    l=[]
    counter =0
    for i in ast.literal_eval(obj):
        if counter!=3:
            l.append(i['name'])
            counter+=1
        else:
            break
    return l    

In [10]:
movies['cast']=movies['cast'].apply(convert3)

In [11]:
def fetch_director(obj):
    l=[]
   
    for i in ast.literal_eval(obj):
        if i['job']=='Director':
            l.append(i['name'])
            
        
            break
    return l  

In [12]:
movies['crew']=movies['crew'].apply(fetch_director)

In [13]:
movies['overview']=movies['overview'].apply(lambda x:x.split())

In [14]:
# sam worthigton
# samworthigton
# sam mendes

In [15]:
movies['genres']=movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords']=movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast']=movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew']=movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [16]:
movies['tags']=movies['overview']+movies['genres']+movies['keywords'] +movies['cast'] + movies['crew']

In [17]:
new_df=movies[['movie_id','title','tags']]

In [18]:
new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x))

C:\Users\vishal haldar\AppData\Local\Temp\ipykernel_1708\487797088.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x))


In [19]:
new_df.head()

,movie_id,title,tags
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...
4,49529,John Carter,"John Carter is a war-weary, former military ca..."


In [20]:
new_df['tags'][0]

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. Action Adventure Fantasy ScienceFiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d SamWorthington ZoeSaldana SigourneyWeaver JamesCameron'

In [21]:
new_df['tags']=new_df['tags'].apply(lambda x:x.lower())

C:\Users\vishal haldar\AppData\Local\Temp\ipykernel_1708\4224080999.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(lambda x:x.lower())


In [22]:
new_df.head()

,movie_id,title,tags
0,19995,Avatar,"in the 22nd century, a paraplegic marine is di..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believed to be dead, ha..."
2,206647,Spectre,a cryptic message from bond’s past sends him o...
3,49026,The Dark Knight Rises,following the death of district attorney harve...
4,49529,John Carter,"john carter is a war-weary, former military ca..."


In [23]:
import nltk

In [24]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [25]:
def stem(text):
    y=[]
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)   
   

In [26]:
new_df['tags']=new_df['tags'].apply(stem)

C:\Users\vishal haldar\AppData\Local\Temp\ipykernel_1708\3514595201.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags']=new_df['tags'].apply(stem)


In [27]:
new_df['tags'][0]

'in the 22nd century, a parapleg marin is dispatch to the moon pandora on a uniqu mission, but becom torn between follow order and protect an alien civilization. action adventur fantasi sciencefict cultureclash futur spacewar spacecoloni societi spacetravel futurist romanc space alien tribe alienplanet cgi marin soldier battl loveaffair antiwar powerrel mindandsoul 3d samworthington zoesaldana sigourneyweav jamescameron'

In [28]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000,stop_words='english')

In [29]:
vectors=cv.fit_transform(new_df['tags']).toarray()

In [30]:
vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [31]:
from sklearn.metrics.pairwise import cosine_similarity

In [32]:
similarity=cosine_similarity(vectors)

In [33]:
sorted(list(enumerate(similarity[0])),reverse=True,key=lambda x:x[1])[1:6]

[(1214, 0.28676966733820225),
 (2405, 0.26901379342448517),
 (3728, 0.2605130246476754),
 (507, 0.255608593705383),
 (539, 0.25038669783359574)]

In [34]:
def recommend(movie):
    movie_index=new_df[new_df['title']==movie].index[0]
    distance=similarity[movie_index]
    movie_list=sorted(list(enumerate(distance)),reverse=True,key=lambda x:x[1])[1:6]
    for i in movie_list:
        print(new_df.iloc[i[0]].title)
        
        

In [35]:
recommend('Batman Begins')

The Dark Knight
Batman
Batman
The Dark Knight Rises
10th & Wolf


In [36]:
import pickle

In [37]:
pickle.dump(new_df,open('movies.pkl','wb'))

In [38]:
new_df['title'].values

array(['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre',
       ..., 'Signed, Sealed, Delivered', 'Shanghai Calling',
       'My Date with Drew'], dtype=object)

In [39]:
pickle.dump(new_df.to_dict(),open('movie_dict.pkl','wb'))

In [40]:
pickle.dump(similarity,open('similarity.pkl','wb'))